# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>
- TAs: Tong Zeng <tozeng@syr.edu>, Priya Matnani <psmatnan@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [27]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [28]:
sms_spam_df = spark.read.csv('/datasets/sms_spam.csv', header=True, inferSchema=True)

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`

In [29]:
# create sms_spam2_df below
# YOUR CODE HERE
sms_spam2_df = sms_spam_df.select((fn.col('type')=='spam').cast('int').alias('type'),fn.col('text'))
sms_spam2_df.show()
#raise NotImplementedError()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [30]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747]
)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [31]:
# create a Pipeline transformer and name it tfidf_pipeline
# YOUR CODE HERE
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
# the tokenizer object
tn = Tokenizer().setInputCol('text').setOutputCol('words')

#the counter vectorizer
cv = CountVectorizer()\
  .setInputCol("words")\
  .setOutputCol("tf")

# tfidf
idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')

# we now create a pipelined transformer
tfidf_pipeline = Pipeline(stages=[tn, cv, idf]).fit(sms_spam2_df)
# raise NotImplementedError()

In [32]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

**(5 pts)** Investigate the fitted pieline above and create a variable `lowest_idf` that contain the set of words with the 5 lowest IDF. **Hint: you must extract the vocabulary from the fitted `CountVectorizer` and the IDF values from the fitted `IDF`, both in the stages of `tfidf_pipeline`. You can put both lists into a Pandas dataframe columns and sort by idf, picking 5 after sorting**

In [33]:
# create lower_idf with list of words with 5 lowest IDF values.
# YOUR CODE HERE
extractvect= [s for s in tfidf_pipeline.stages if isinstance(s, feature.CountVectorizerModel)]
vocab = [v.vocabulary for v in extractvect]
#print(vocab)
model_idf= [s for s in tfidf_pipeline.stages if isinstance(s, feature.IDFModel)]
value_idf =[v.idf for v in model_idf]
#print(value_idf)
v_df= pd.DataFrame ({'vocab': value for value in vocab})
#print(v_df)
r_df = pd.DataFrame.from_records(value_idf)
r_df= r_df.T
r_df.columns =['idf']
#print(r_df)
f_df= v_df.join(r_df)
f_df.sort_values('idf')
lowest_idf=f_df.vocab[0:5]
lowest_idf= set(lowest_idf)
#raise NotImplementedError()

In [34]:
# (5 pts)
# it is a set
np.testing.assert_equal(type(lowest_idf), set)
# it has 5 elements
np.testing.assert_equal(len(lowest_idf), 5)
# each element is a string
np.testing.assert_equal({type(w) for w in lowest_idf}, {str})

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [35]:
# create sms_spam3_df below
# YOUR CODE HERE
sms_spam3_df = sms_spam2_df.select(fn.col('type'),fn.col('text'),
                                   ((fn.length(fn.regexp_extract(fn.col('text'),'[A-Z]{3,}',0)))>2).cast('int').alias('has_uppercase'))
sms_spam3_df.show()
#raise NotImplementedError()

+----+--------------------+-------------+
|type|                text|has_uppercase|
+----+--------------------+-------------+
|   0|Go until jurong p...|            0|
|   0|Ok lar... Joking ...|            0|
|   1|Free entry in 2 a...|            0|
|   0|U dun say so earl...|            0|
|   0|Nah I don't think...|            0|
|   1|FreeMsg Hey there...|            0|
|   0|Even my brother i...|            0|
|   0|As per your reque...|            0|
|   1|WINNER!! As a val...|            1|
|   1|Had your mobile 1...|            1|
|   0|I'm gonna be home...|            0|
|   1|SIX chances to wi...|            1|
|   1|URGENT! You have ...|            1|
|   0|I've been searchi...|            0|
|   0|I HAVE A DATE ON ...|            1|
|   1|XXXMobileMovieClu...|            1|
|   0|Oh k...i'm watchi...|            0|
|   0|Eh u remember how...|            0|
|   0|Fine if that's th...|            0|
|   1|England v Macedon...|            1|
+----+--------------------+-------

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [36]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]

In [37]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [38]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [39]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3349, 1674, 551]

**(5 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$). Fit those pipelines to the appropriate data split.

1. Logistic regression with $\lambda=0$ and $\alpha=0$ (assign the fitted pipeline to `lr_pipeline1`)
2. Logistic regression with $\lambda=0.02$ and $\alpha=0.2$ (assign the fitted pipeline to `lr_pipeline2`)
3. Logistic regression with $\lambda=0.1$ and $\alpha=0.4$ (assign the fitted pipeline to `lr_pipeline3`)

In [40]:
# create lr_pipeline1, lr_pipeline2, and lr_pipeline3
# YOUR CODE HERE
from pyspark.ml.classification import LogisticRegression
regParam1 = 0
regParam2 = 0.02
regParam3 = 0.1
elasticNetParam1 = 0
elasticNetParam2 = 0.2
elasticNetParam3 = 0.4

lr1 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(regParam1).\
    setElasticNetParam(elasticNetParam1)
lr_pipeline1 = Pipeline(stages=[tfidf_pipeline, lr1]).fit(training_df)

lr2 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(regParam2).\
    setElasticNetParam(elasticNetParam2)
lr_pipeline2 = Pipeline(stages=[tfidf_pipeline, lr2]).fit(training_df)

lr3 = LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(regParam3).\
    setElasticNetParam(elasticNetParam3)
lr_pipeline3 = Pipeline(stages=[tfidf_pipeline, lr3]).fit(training_df)

#raise NotImplementedError()

In [41]:
# (5 pts)
np.testing.assert_equal(type(lr_pipeline1), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline2), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline3), pipeline.PipelineModel)
np.testing.assert_array_equal([type(s) for s in lr_pipeline1.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline2.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline3.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

**(5 pts)** Use the evaluator object defined below to compute the area under the curve of your predictors. For example, to compute the area under the curve of pipeline 1 for a dataframe `df`, you would run

```python
evaluator.evaluate(lr_pipeline1.transform(df))
```

Assign the AUC of the three models to the variables `AUC1`, `AUC2`, and `AUC3`, and and assign the pipeline with the best model to a variable `best_model`

In [42]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='type')

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [43]:
# print the AUC for the three models as follows
# print("Model 1 AUC: ", evaluator.evaluate(....))
AUC1 = evaluator.evaluate(lr_pipeline1.transform(validation_df))
print("Model 1 AUC: ", evaluator.evaluate(lr_pipeline1.transform(validation_df)))
AUC2 = evaluator.evaluate(lr_pipeline2.transform(validation_df))
print("Model 2 AUC: ", evaluator.evaluate(lr_pipeline2.transform(validation_df)))
AUC3 = evaluator.evaluate(lr_pipeline3.transform(validation_df))
print("Model 3 AUC: ", evaluator.evaluate(lr_pipeline3.transform(validation_df)))

# finally, based on these, assign the best validated 
# By observing the Area under the curve, I conclude that the best model is AUC2 i.e. lr_pipeline2
best_model = lr_pipeline2
# model to a variable best_model
# YOUR CODE HERE
# raise NotImplementedError()

Model 1 AUC:  0.9557218489415972
Model 2 AUC:  0.9871555611031791
Model 3 AUC:  0.9686667539402503


In [44]:
# (5 pts)
np.testing.assert_array_equal([type(AUC1), type(AUC2), type(AUC3)],
                             [float, float, float])
# AUC less than 1
np.testing.assert_array_less([AUC1, AUC2, AUC3], [1, 1, 1])
# AUC more than 0.5
np.testing.assert_array_less([.5, .5, .5],
                            [AUC1, AUC2, AUC3])

# Question 2.5: Choose best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [45]:
# assign to AUC_best the AUC of the best model selected before
# YOUR CODE HERE

AUC_best = best_model.\
    transform(testing_df).\
    select(fn.avg(fn.expr('float(prediction=type)')).alias('accuracy')).first()['accuracy']

AUC_best

#raise NotImplementedError()

0.969147005444646

In [46]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 
                               0.976126746201693, significant=2)

# Question 2.6: Inference

Use the pipeline 2 fitted above (`lr_pipeline2`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [47]:
# create positive_words and negative_words pandas dataframe below
# YOUR CODE HERE
vocabulary = lr_pipeline2.stages[0].stages[1].vocabulary
weights = lr_pipeline2.stages[-1].coefficients.toArray()
coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})
positive_words = coeffs_df.sort_values('weight', ascending=False).head(20)
negative_words = coeffs_df.sort_values('weight', ascending=True).head(20)
#raise NotImplementedError()

The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.590870</td>
    </tr>
    <tr>
      <th>12237</th>
      <td>08714712388</td>
      <td>0.533567</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.517100</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.513278</td>
    </tr>
    <tr>
      <th>9064</th>
      <td>gbp/sms</td>
      <td>0.468274</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.162493</td>
    </tr>
    <tr>
      <th>2444</th>
      <td>fighting</td>
      <td>-0.060939</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.059061</td>
    </tr>
  </tbody>
</table>

In [48]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.3701485692317927, significant=2)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.6661952507442954, significant=2)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

In [ ]:
# your code and comments below
# YOUR CODE HERE

tn = Tokenizer().setInputCol('text').setOutputCol('words')

#the counter vectorizer
cv = CountVectorizer()\
  .setInputCol("words")\
  .setOutputCol("tf")

# tfidf
idf = IDF().\
    setInputCol('tf').\
    setOutputCol('tfidf')

va = feature.VectorAssembler(inputCols = ['has_uppercase','tfidf'], outputCol = 'features')

mAS = feature.MaxAbsScaler().\
    setInputCol('features').\
    setOutputCol('output')

# we now create a pipelined transformer
tfidf_pipeline = Pipeline(stages=[tn, cv, idf, va, mAS]).fit(sms_spam3_df)

lr = LogisticRegression().setLabelCol('type').setFeaturesCol('output').setRegParam(0.2).setElasticNetParam(0.1)
lr_pipeline = Pipeline(stages=[tfidf_pipeline, lr]).fit(sms_spam3_df)

lr_pipeline.stages[-1].coefficients


In [ ]:

# 1. is has_uppercase a feature that is positively or negative related to an SMS being spam?
# ANS: the has_uppercase is positively related to an SMS being spam as the coefficient is positive
# 2. what is the ratio of the coefficient of has_uppercase to the biggest positive tfidf coefficient?
lr_pipeline.stages[-1].coefficients[0]/max(lr_pipeline.stages[-1].coefficients)
# raise NotImplementedError()